In [4]:
import pickle

tensor(50)


In [5]:
# class DotDict(dict):
#     def __init__(self, **kwds):
#         self.update(kwds)
#         self.__dict__ = self

# path_sbm_train = "data/SBM/SBM_CLUSTER_train.pkl"
# path_sbm_val = "data/SBM/SBM_CLUSTER_val.pkl"
# path_sbm_test = "data/SBM/SBM_CLUSTER_test.pkl"
# with open(path_sbm_train, 'rb') as f:
#     sbm_train = pickle.load(f)
# with open(path_sbm_val, 'rb') as f:
#     sbm_val = pickle.load(f)
# with open(path_sbm_test, 'rb') as f:
#     sbm_test = pickle.load(f)
# # sbm_train

# # for tmp in sbm_train:
# #     print(len(tmp.node_feat))
# # sbm_train[0].node_feat
# # edge_index = torch.tensor([[],[]])
# # print(edge_index)
# # print(torch.cat((edge_index, torch.nonzero(sbm_train[0].W).T+100), 1))
# # x = torch.BoolTensor([0]*10)
# # x[1:3] = True
# # print(x)
# # print([1,2]*2)
# s_train = 0
# for tmp in sbm_train:
#     s_train += len(tmp.node_feat)
# s_val = 0
# for tmp in sbm_val:
#     s_val += len(tmp.node_feat)
# s_test = 0
# for tmp in sbm_test:
#     s_test += len(tmp.node_feat)
# print(s_train, s_val, s_test, s_train + s_val+s_test)

In [1]:
# from torch_geometric.data import Data
# K=1000
# NBTRAIN = len(sbm_train)
# NBVAL = len(sbm_val)
# NBTEST = len(sbm_test)

# import torch
# for k in range(K):
#     print(k)
#     idx_base=0
#     edge_index = torch.LongTensor([[],[]])
#     x = torch.tensor([])
#     y = torch.LongTensor([])
#     for idx_graph in range(k*int(NBTRAIN/K),(k+1)*int(NBTRAIN/K)):
#         sub_graph = sbm_train[idx_graph]
#         edge_index = torch.cat(
#             (edge_index, torch.nonzero(sub_graph.W).T+idx_base), 1)
#         x = torch.cat((x, torch.unsqueeze(sub_graph.node_feat, 1)), 0)
#         y = torch.cat((y, sub_graph.node_label), 0)
#         idx_base += len(sub_graph.node_feat)
#     idx_train = idx_base
#     for idx_graph in range(k*int(NBVAL/K), (k+1)*int(NBVAL/K)):
#         sub_graph = sbm_val[idx_graph]
#         edge_index = torch.cat(
#             (edge_index, torch.nonzero(sub_graph.W).T+idx_base), 1)
#         x = torch.cat((x, torch.unsqueeze(sub_graph.node_feat, 1)), 0)
#         y = torch.cat((y, sub_graph.node_label), 0)
#         idx_base += len(sub_graph.node_feat)
#     idx_val = idx_base
#     for idx_graph in range(k*int(NBTEST/K), (k+1)*int(NBTEST/K)):
#         sub_graph = sbm_test[idx_graph]
#         edge_index = torch.cat(
#             (edge_index, torch.nonzero(sub_graph.W).T+idx_base), 1)
#         x = torch.cat((x, torch.unsqueeze(sub_graph.node_feat, 1)), 0)
#         y = torch.cat((y, sub_graph.node_label), 0)
#         idx_base += len(sub_graph.node_feat)
#     idx_test = idx_base

#     x = torch.zeros(x.shape[0], 7).scatter(1, torch.tensor(x,dtype=torch.int64), 1)[:, 1:]
#     train_mask = torch.BoolTensor([0]*idx_base)
#     train_mask[0:idx_train] = True
#     val_mask = torch.BoolTensor([0]*idx_base)
#     val_mask[idx_train:idx_val] = True
#     test_mask = torch.BoolTensor([0]*idx_base)
#     test_mask[idx_val:idx_test] = True
#     # with open("data/SBM/{}_uncopynode.pkl".format(k), "wb") as f:
#     #     pickle.dump(Data(edge_index=edge_index, x = x, y = y, train_mask = train_mask, val_mask = val_mask, test_mask = val_mask), f)


In [25]:
from models import Gat,Sage
from torch import optim
import pickle
import torch.nn.functional as F
import utils
import torch.nn as nn
import torch_geometric.nn as gnn


# model = Gat(6,6).to('cuda:0')
datas = []
for i in range(1000):
    path = f'data/SBM/{i}_uncopynode.pkl'
    with open(path, 'rb') as f:
        datas.append(pickle.load(f).to('cuda:0'))


In [33]:
# class Gcn(nn.Module):
#     def __init__(self, nfeat, nclass):
#         super(Gcn, self).__init__()
#         hdim = 146
#         self.lnin = nn.Linear(nfeat,hdim)
#         self.gcn = gnn.GCNConv(hdim,hdim)
#         self.gcn1 = gnn.GCNConv(nfeat, hdim)
#         # self.gcn2 = gnn.GCNConv(hdim, hdim)
#         # self.gcn3 = gnn.GCNConv(hdim, hdim)
#         self.gcn4 = gnn.GCNConv(hdim, nclass)
#         self.lnout = nn.Linear(hdim, nclass)
#         return

#     def forward(self, x, edge_index):
#         # x = self.gcn1(x, edge_index)
#         # x = F.relu(x)
#         # x = self.gcn2(x, edge_index)
#         # x = F.relu(x)
#         # x = self.gcn3(x, edge_index)
#         # x = F.relu(x)
#         # x = self.gcn4(x, edge_index)
#         x = self.lnin(x)
#         x = self.gcn(x,edge_index)
#         x = F.relu(x)
#         x = self.lnout(x)
#         return x
from models import Appnp,Sage
model = Sage(6,6).to('cuda:0')
model.train()

# optimizer = optim.SGD(
#     model.parameters(), lr=0.1,momentum=0.9, weight_decay=1e-6)
optimizer = optim.Adam(
    model.parameters(), lr=0.001, weight_decay=1e-6)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='min',factor=0.5,patience=5)
for epoch in range(1000):
    losses = 0
    accus = 0
    for iter, data in enumerate(datas):
        preds = model(data.x, data.edge_index)
        loss = F.cross_entropy(preds[data.train_mask],data.y[data.train_mask])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses += loss.item()
        accus += utils.accuracy(preds[data.train_mask],
                                data.y[data.train_mask])
    print('Iter{}, loss:{},train_acc:{}'.format(epoch+1, losses/(iter+1), accus/(iter+1)))
    scheduler.step(losses/(iter+1))
    # break


/home/user1/anaconda3/envs/test/lib/python3.8/site-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Iter1, loss:1.7551698240041733,train_acc:0.23264149629279077
Iter2, loss:1.6739285281896592,train_acc:0.33090114494790884
Iter3, loss:1.6265976947546006,train_acc:0.36335642236178745
Iter4, loss:1.5885578683614732,train_acc:0.3740679216735574
Iter5, loss:1.5606922438144684,train_acc:0.384665871952323
Iter6, loss:1.544199174642563,train_acc:0.39441720376530964
Iter7, loss:1.5356242880821227,train_acc:0.4027025862625091
Iter8, loss:1.5308883451223374,train_acc:0.40906894432731555
Iter9, loss:1.5275925365686416,train_acc:0.41423943182294454


KeyboardInterrupt: 